# Importing Dependencies

In [83]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Dropout , BatchNormalization,Masking,Bidirectional
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard

In [8]:
signs = pd.read_csv('signs.csv')

In [25]:
signs

,Sign_id,Sign_Name
0,1,Hello
1,2,Bye
2,3,Morning
3,4,Good
4,5,Nice
5,6,House
6,7,Thank You
7,8,Welcome
8,9,Yes
9,10,No


In [86]:
classes = np.array(signs["Sign_Name"])

In [87]:
classes = np.insert(classes,0,['no prediction'])

In [88]:
classes

array(['no prediction', 'Hello', 'Bye', 'Morning', 'Good', 'Nice',
       'House', 'Thank You', 'Welcome', 'Yes', 'No', 'Work'], dtype=object)

# Preprocessing

In [ ]:
# # Function to extract features from a video by breaking it into frames
# def extract_features_from_video(inception_model,video_path, frame_size=(299, 299)):
#     cap = cv2.VideoCapture(video_path)
#     features = []
    
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break
#         # Resize the frame to the required size for InceptionResNetV2
#         frame_resized = cv2.resize(frame, frame_size)
#         img_array = img_to_array(frame_resized)
#         img_array = np.expand_dims(img_array, axis=0)
#         img_array = preprocess_input(img_array)
#         # Extract the feature from the frame
#         feature = inception_model.predict(img_array)
#         features.append(feature)
    
#     cap.release()
#     return np.array(features)

In [2]:
import cv2
import numpy as np

# Function to uniformly sample 20 frames from a video
def uniform_sample_video(video_path, target_frame_count=20,IMAGE_SIZE = (299,299)):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []

    if total_frames <= target_frame_count:
        # If the video has fewer or equal to 20 frames, just return all frames with padding if necessary
        success, frame = cap.read()
        while success:
            frame = cv2.resize(frame, IMAGE_SIZE)  # Resize to match the model input size
            frames.append(frame)
            success, frame = cap.read()
        cap.release()

        # Pad with black frames if fewer than 20 frames
        while len(frames) < target_frame_count:
            frames.append(np.zeros((IMAGE_SIZE[0], IMAGE_SIZE[1], 3)))
        
        return np.array(frames)
    
    # Otherwise, we will uniformly sample 20 frames
    frame_indices = np.linspace(0, total_frames - 1, target_frame_count).astype(int)
    
    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)  # Go to the specific frame index
        success, frame = cap.read()
        if success:
            frame = cv2.resize(frame, IMAGE_SIZE)  # Resize to match the model input size
            frames.append(frame)

    cap.release()
    
    return np.array(frames)


In [3]:
# Function to extract features from a video by breaking it into frames
def extract_features_from_frames(inception_model,frames, frame_size=(299, 299)):
    features = []
    for frame in frames:
        # Resize the frame to the required size for InceptionResNetV2
        frame_resized = cv2.resize(frame, frame_size)
        img_array = img_to_array(frame_resized)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        # Extract the feature from the frame
        feature = inception_model.predict(img_array)
        features.append(feature)
    return np.array(features)


# Function to traverse the directory and process videos
def process_videos(videos_dir,inception_model):
    X = []  # To store stacked features for each video
    y = []  # To store the corresponding labels
    # Traverse the directories
    for label in os.listdir(videos_dir):  # Labels are the directory names (0-11)
        label_path = os.path.join(videos_dir, label)
        
        if os.path.isdir(label_path):  # Check if it's a directory
            if label == '0':
                # If it's class '0', videos are directly inside
                for video_file in os.listdir(label_path):
                    video_path = os.path.join(label_path, video_file)
                    if video_file.endswith(('.mp4', '.avi', '.mov')):
                        #convert the frames to 20 uniformly
                        frames = uniform_sample_video(video_path)
                        # Extract features from the video
                        video_features = extract_features_from_frames(inception_model=inception_model,frames=frames)
                        X.append(video_features)  # Store features
                        y.append(int(label))  # Store label
            else:
                # For classes 1-11, we have subdirectories for each person
                for person in os.listdir(label_path):
                    person_path = os.path.join(label_path, person)
                    if os.path.isdir(person_path):
                        for video_file in os.listdir(person_path):
                            video_path = os.path.join(person_path, video_file)
                            if video_file.endswith(('.mp4', '.avi', '.mov')):  # Adjust as needed
                                #convert the frames to 20 uniformly
                                frames = uniform_sample_video(video_path)
                                # Extract features from the video
                                video_features = extract_features_from_frames(inception_model=inception_model,frames=frames)
                                X.append(video_features)  # Store features
                                y.append(int(label))  # Store label
    return X, y


In [4]:
# Shuffle and split dataset
def shuffle_and_split_dataset(X, y, test_size=0.3, random_state=42):
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    return X_train, X_test, y_train, y_test

# Model Creation

In [32]:
def create_lstm_gru_model(input_shape, num_classes):
    model = Sequential()

    # Masking layer to handle padded inputs (optional if using padded sequences)
    model.add(Masking(mask_value=0.0, input_shape=input_shape))

    # Dropout to prevent overfitting
    model.add(Dropout(0.5))

    # First Bidirectional LSTM pair
    model.add(Bidirectional(LSTM(1536, return_sequences=True, dropout=0.3, kernel_regularizer='l2')))
    model.add(Bidirectional(LSTM(1536, return_sequences=True, dropout=0.3, kernel_regularizer='l2')))

    # Second Bidirectional GRU pair
    model.add(GRU(1024, return_sequences=True, dropout=0.3, kernel_regularizer='l2'))
    model.add(GRU(1024, return_sequences=True, dropout=0.3, kernel_regularizer='l2'))

    # Third Bidirectional LSTM pair
    model.add(Bidirectional(LSTM(768, return_sequences=True, dropout=0.3, kernel_regularizer='l2')))
    model.add(Bidirectional(LSTM(768, return_sequences=True, dropout=0.3, kernel_regularizer='l2')))

    # Fourth GRU (single vector output, hence return_sequences=False)
    model.add(GRU(512, return_sequences=False, dropout=0.3, kernel_regularizer='l2'))

    # Dense layer
    model.add(Dense(256, activation='relu', kernel_regularizer='l2'))

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate = 0.0001), metrics=['accuracy'])

    return model


# Callbacks

In [6]:
# Set up directories if they dont exist for saving models and logs

os.makedirs('saved_models', exist_ok=True)
os.makedirs('logs', exist_ok=True)

In [7]:
checkpoint_dir = 'saved_models'
log_dir = 'logs'

In [8]:
# Callback to save the best model based on validation accuracy in HDF5 format (.h5)
save_best_model = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'best_model.keras'),
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Callback to save the model every 25 epochs in HDF5 format (.h5)
class CustomModelCheckpoint(ModelCheckpoint):
    def __init__(self, *args, save_every_n_epochs=25, **kwargs):
        super().__init__(*args, **kwargs)
        self.save_every_n_epochs = save_every_n_epochs

    def on_epoch_end(self, epoch, logs=None):
        # Only save every `save_every_n_epochs` epochs
        if (epoch + 1) % self.save_every_n_epochs == 0:
            super().on_epoch_end(epoch, logs)

save_every_25_epochs = CustomModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'model_epoch_{epoch:02d}.keras'),
    monitor='val_accuracy',  # Monitor any metric you want (accuracy or loss)
    verbose=1,
    save_every_n_epochs=25
)

# Early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

# Reduce learning rate on plateau callback
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=5,
    verbose=1,
    mode='min',
    min_lr=1e-6
)

# TensorBoard callback for logging
tensorboard = TensorBoard(
    log_dir='logs',
    histogram_freq=1,
    write_graph=True,
    write_images=True
)

# Combine all callbacks
callbacks = [save_best_model, save_every_25_epochs, early_stopping, reduce_lr, tensorboard]

# Sava and Load function for feature data

In [15]:
# Function to save X and y only if they don't exist
def save_features_labels(X, y, X_filepath='X_features.npy', y_filepath='y_labels.npy'):
    if not os.path.exists(X_filepath):
        X_array = np.array(X)
        np.save(X_filepath, X_array)
        print(f"X saved to {X_filepath}")
    else:
        print(f"{X_filepath} already exists. Skipping saving X.")

    if not os.path.exists(y_filepath):
        y_array = np.array(y)
        np.save(y_filepath, y_array)
        print(f"y saved to {y_filepath}")
    else:
        print(f"{y_filepath} already exists. Skipping saving y.")

In [57]:
# Function to load X and y features that have been preprocessed and saved
def load_features_labels(X_filepath='X_features.npy', y_filepath='y_labels.npy'):
    if os.path.exists(X_filepath) and os.path.exists(y_filepath):
        X = np.load(X_filepath, allow_pickle=True)
        y = np.load(y_filepath, allow_pickle=True)
        print(f"Loaded X from {X_filepath} and y from {y_filepath}")
        return X, y
    else:
        print("Files do not exist. You need to extract and save features first.")
        return None, None

# Training

1.Complete training in one shot

In [ ]:
# Load the InceptionResNetV2 model for feature extraction (without the top classification layer)
inception_model = InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg')

videos_dir = 'videos'
X,y = process_videos(videos_dir=videos_dir,inception_model=inception_model)

# Convert to numpy arrays if needed
X = np.array(X,dtype=object)
y = np.array(y)

# Pad the sequences
X_padded = pad_sequences(X, padding='post', dtype='float32')

# Reshape to remove the extra dimension (from (batch_size, time_steps, 1, 1536) to (batch_size, time_steps, 1536))
X_reshaped = np.reshape(X_padded, (X_padded.shape[0], X_padded.shape[1], X_padded.shape[3]))

#save the preprocessed X and Y for training later
save_features_labels(X_reshaped, y, X_filepath='X_features_anyframes.npy', y_filepath='y_labels_anyframes.npy')

# Load X and y features from .npy files
X_loaded,y_loaded = load_features_labels(X_filepath='X_features_anyframes.npy', y_filepath='y_labels_anyframes.npy')

#training from scratch
X_train, X_test, y_train, y_test = shuffle_and_split_dataset(X_loaded, y_loaded)

input_shape = (None, 1536)  # dynamic frames, each with 1536 features
num_classes = 12  #  number of classes
model = create_lstm_gru_model2(input_shape, num_classes)

history = model.fit(X_train, y_train, callbacks = callbacks,validation_data=(X_test, y_test), epochs=100, batch_size=16,verbose=1)

2.Training is divided into parts,each cell is doing its part

In [17]:
# Load the InceptionResNetV2 model for feature extraction (without the top classification layer)
inception_model = InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg')

videos_dir = 'videos'
X,y = process_videos(videos_dir=videos_dir,inception_model=inception_model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [51]:
# Convert to numpy arrays if needed
X = np.array(X,dtype = object)
y = np.array(y)

# Pad the sequences
X_padded = pad_sequences(X, padding='post', dtype='float32')

# Reshape to remove the extra dimension (from (batch_size, time_steps, 1, 1536) to (batch_size, time_steps, 1536))
X_reshaped = np.reshape(X_padded, (X_padded.shape[0], X_padded.shape[1], X_padded.shape[3]))

# save_features_labels(X_reshaped, y, X_filepath='X_features_anyframes.npy', y_filepath='y_labels_anyframes.npy')
save_features_labels(X_reshaped, y, X_filepath='X_features_20uniformframes.npy', y_filepath='y_labels_20uniformframes.npy')

X saved to X_features_20uniformframes.npy
y saved to y_labels_20uniformframes.npy


In [58]:
X_padded.shape

(838, 20, 1, 1536)

In [58]:
# Load X and y features from .npy files
X_loaded,y_loaded = load_features_labels(X_filepath='X_features_20uniformframes.npy', y_filepath='y_labels_20uniformframes.npy')

Loaded X from X_features_20uniformframes.npy and y from y_labels_20uniformframes.npy


In [59]:
len(X_loaded)

838

In [63]:
#shuffle and split into train and test datasets
X_train, X_test, y_train, y_test = shuffle_and_split_dataset(X_loaded, y_loaded)

In [21]:
#training from scratch
input_shape = (None, 1536)  # dynamic(varying) frames, each with 1536 features
num_classes = 12  #  number of classes
model = create_lstm_gru_model(input_shape, num_classes)

history = model.fit(X_train, y_train,callbacks = callbacks,validation_data=(X_test,y_test), epochs=100, batch_size=16,verbose=1)

c:\Users\mvswa\dynamic_signlanguage_detect\signLD\Lib\site-packages\keras\src\layers\core\masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 26s/step - accuracy: 0.1075 - loss: 74.9067 
Epoch 1: val_accuracy improved from -inf to 0.08333, saving model to saved_models\best_model.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 1320s 27s/step - accuracy: 0.1074 - loss: 74.2419 - val_accuracy: 0.0833 - val_loss: 9.7070 - learning_rate: 0.0010
Epoch 2/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 26s/step - accuracy: 0.1039 - loss: 8.1950 
Epoch 2: val_accuracy did not improve from 0.08333
48/48 ━━━━━━━━━━━━━━━━━━━━ 1289s 27s/step - accuracy: 0.1041 - loss: 8.1684 - val_accuracy: 0.0476 - val_loss: 4.6807 - learning_rate: 0.0010
Epoch 3/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 26s/step - accuracy: 0.1037 - loss: 4.2428 
Epoch 3: val_accuracy did not improve from 0.08333
48/48 ━━━━━━━━━━━━━━━━━━━━ 1292s 27s/step - accuracy: 0.1038 - loss: 4.2430 - val_accuracy: 0.0714 - val_loss: 7.4272 - learning_rate: 0.0010
Epoch 4/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 26s/step - accuracy: 0.1386 - loss: 7.4307 
Epoch 4: val_accuracy did n

In [31]:
# Save the trained model
model.save('saved_models\\manually_saved_model.h5')

In [50]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_1 (Masking)             │ (None, None, 1536)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, None, 1536)     │    18,880,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 1536)           │    14,164,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       786,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 12)             │         6,156 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,838,604 (129.08 MB)

 Trainable params: 33,838,604 (129.08 MB)

 Non-trainable params: 0 (0.00 B)

# MODEL TESTING

In [102]:
trained_model = load_model('saved_models\\004_model_94perc_acc.keras')

In [61]:
trained_model.evaluate(X_test,y_test)

8/8 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.9746 - loss: 0.6635


[0.6933056712150574, 0.9603174328804016]

In [64]:
y_prob = trained_model.predict(X_test)

8/8 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step


In [65]:
y_test_mod = np.argmax(y_test,axis = 1)

In [66]:
y_pred = np.argmax(y_prob,axis = 1)

In [81]:
print(y_test_mod)
print()
print(y_pred)

[10  0  0  1  6  9  5  6  5  2  1  4  9  8  2 10  2  4  3  4  4  1  0  6
  6 11  6  2  3  3  3  4 10  8  6  8  3  6 11 10  4  4  5 10  0  2  8  6
  4  4  0  6 11  2 11  7  1  9  7  6  7  3  2 10  0  9  8 10  6  2  4  0
  4  5  5  1 11  1  9  1  2  4  9  5 11 10  4  2  2 11  3  1 10  4  7 11
  7  4  5  3  3 11 11  2  1  5  2 11  8 10  8  6  2  4  8  7  5  4 11  5
 11 11  5  5  4 11  7  6  0 11  5  8  6  5  2  4  7  2  9  7  2  4  5  7
  1  0  4  5 11  6  7  3 11  7  8  5  7 11  8  4 10  5  5  3  6  9  0  7
  4 11  9  5 10  3  6  3  5  5  2  6  3 11  5  4  7  8  4  1  2  4  4  8
  7  7 10  4 10  8  3  5  5  6 11  9  0  3  4  3  6  3  7  6  8  6  5  2
  9  6 10 10 10  3  9  1  8  7  6  4  4  8  5  0  6  5 11  8  1  2  8  5
  0  3  4 11  1  4 10  5  6  4  4  5]

[10  0  0  1  6  9  5  6  5  2  1  4  9  8  2 10  2  4  3  4  4  1  0  6
  6 11  6  2  3  3  3  4 10  8  6  8  3  6 11 10  4  4  8 10  0  2  8  6
  4  4  0  6 11  2 11  7  1  9  7  6  7  3  2 10  0  9  8 10  6  9  4  0
  4  5  5  1

In [67]:
from sklearn.metrics import accuracy_score

# Assuming y_test is the actual labels and y_pred is the model's predictions
accuracy = accuracy_score(y_test_mod, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 98.02%


In [62]:
trained_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_2 (Masking)             │ (None, 20, 1536)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 20, 1536)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_6 (Bidirectional) │ (None, 20, 3072)       │    37,761,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_7 (Bidirectional) │ (None, 20, 3072)       │    56,635,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_4 (GRU)                     │ (None, 20, 1024)       │    12,589,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ (None, 20, 1024)       │     6,297,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_8 (Bidirectional) │ (None, 20, 1536)       │    11,016,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_9 (Bidirectional) │ (None, 20, 1536)       │    14,161,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_6 (GRU)                     │ (None, 512)            │     3,148,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 12)             │         3,084 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 425,233,190 (1.58 GB)

 Trainable params: 141,744,396 (540.71 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 283,488,794 (1.06 GB)

# Real Time Detection

In [35]:
inception_model = InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg')

In [34]:
trained_model = load_model('saved_models\\004_model_94perc_acc.keras')

In [99]:
# Function to extract features from a single frame using the feature extractor
def extract_frame_features(frame,inception_model,frame_size = (299,299)):
    # Resize the frame to the required size for InceptionResNetV2
    frame_resized = cv2.resize(frame, frame_size)
    img_array = img_to_array(frame_resized)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    # Extract the feature from the frame
    feature = inception_model.predict(img_array)
    return feature

# Uniform sampling to reduce frames to SEQUENCE_LENGTH
def uniform_sample_frames(frames, target_frame_count=20):
    total_frames = len(frames)
    if total_frames <= target_frame_count:
        return frames  # Return all frames if fewer than target_frame_count

    frame_indices = np.linspace(0, total_frames - 1, target_frame_count).astype(int)
    return [frames[i] for i in frame_indices]

# Real-time sign language detection function using the webcam
def real_time_sign_detection(model, inception_model, classes,sequence_length=20, max_frames=25):
    gesture_label = ''
    cap = cv2.VideoCapture(0)  # Open the webcam
    feature_buffer = []  # Buffer for the extracted features

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Extract features from the frame
        frame_features = extract_frame_features(frame, inception_model)
        feature_buffer.append(frame_features)

        # Keep only the last 'max_frames' features
        if len(feature_buffer) > max_frames:
            feature_buffer = feature_buffer[-max_frames:]

        # If we have enough frames (MAX_FRAMES), perform uniform sampling to get SEQUENCE_LENGTH features
        if len(feature_buffer) == max_frames:
            sampled_features = uniform_sample_frames(feature_buffer, target_frame_count=sequence_length)
            # input_features = np.array([sampled_features])  # Add batch dimension
            
            # Pad the sequences
            padded_sequences = pad_sequences(sampled_features, padding='post', dtype='float32')
            print(padded_sequences.shape)
            # Reshape to remove the extra dimension (from (batch_size, time_steps, 1, 1536) to (batch_size, time_steps, 1536))
            reshaped_sequences = np.reshape(padded_sequences, (1,padded_sequences.shape[0], padded_sequences.shape[2]))

            # Perform gesture prediction using the LSTM-GRU model
            prediction = model.predict(reshaped_sequences)
            predicted_class = np.argmax(prediction[0])
            feature_buffer = []

            # Display the prediction on the screen
            gesture_label = f"Predicted Gesture: {classes[predicted_class]}"
            cv2.putText(frame, "SIGN LANGUAGE DETECTION", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 2)
            cv2.putText(frame, gesture_label, (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
            
        if(len(gesture_label)>0):
            cv2.putText(frame, "SIGN LANGUAGE DETECTION", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 2)
            cv2.putText(frame, gesture_label, (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)

        # Show the frame
        cv2.imshow('Real-Time Sign Language Detection', frame)
        
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [90]:
classes

array(['no prediction', 'Hello', 'Bye', 'Morning', 'Good', 'Nice',
       'House', 'Thank You', 'Welcome', 'Yes', 'No', 'Work'], dtype=object)

In [100]:
real_time_sign_detection(model = trained_model, inception_model = inception_model, classes = classes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
(20, 1, 1536)
1/1 ━━━━━━━

# Experimenting

In [9]:
inception_model = InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg')
inception_model.summary()

Model: "inception_resnet_v2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, None,      │        864 │ input_layer[0][0] │
│                     │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, None,      │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, None,      │          0 │ batch_normalizat… │
│ (Activation)        │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, None,      │      9,216 │ activation[0][0]  │
│                     │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None,      │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, None,      │          0 │ batch_normalizat… │
│ (Activation)        │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, None,      │     18,432 │ activation_1[0][… │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None,      │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, None,      │          0 │ batch_normalizat… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, None,      │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, None,      │      5,120 │ max_pooling2d[0]… │
│                     │ None, 80)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None,      │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ None, 80)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, None,      │          0 │ batch_normalizat… │
│ (Activation)        │ None, 80)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, None,      │    138,240 │ activation_3[0][… │
│                     │ None, 192)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None,      │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ None, 192)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, None,      │          0 │ batch_normalizat

 Total params: 54,336,736 (207.28 MB)

 Trainable params: 54,276,192 (207.05 MB)

 Non-trainable params: 60,544 (236.50 KB)